In [ ]:
pip install keras_tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras_tuner.tuners import Hyperband
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [ ]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=['EIN', 'NAME'])

In [ ]:
# Determine the number of unique values in each column.
df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = df['APPLICATION_TYPE'].value_counts()
print(app_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#1000 seemed to compress the data too much, 700 left enough similar types without low value outliers
application_types_to_replace = list(app_type_counts[app_type_counts < 700].index)

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#same as APPLICATION_TYPE above re: 1000 v 750
classifications_to_replace = list(classification_counts[classification_counts < 750].index)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

**DATA ANALYSIS FOR OPTIMIZATION**

APPLICATION TYPE (binned)</br>
CLASSIFICATION (binned)</br>
AFFILIATION</br>
USE_CASE</BR>
ORGANIZATION</BR>
STATUS (dropped)</BR>
INCOME_AMT</BR>
SPECIAL_CONSIDERATIONS (dropped)</BR>
ASK_AMT (binned)</BR>


In [ ]:
#AFFILIATION - Affiliated sector of industry
print(df['AFFILIATION'].value_counts().head(10))

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64


In [ ]:
# List of values to be binned as 'Other'
#other_affiliations = ['Family/Parent', 'National', 'Regional', 'Other']

# Use the `replace` function to bin the desired values
#df['AFFILIATION'] = df['AFFILIATION'].replace(other_affiliations, 'Other')

#drop all values other than independent and companysponsored
df = df[df['AFFILIATION'].isin(['Independent', 'CompanySponsored'])]

# Verify the changes
print(df['AFFILIATION'].value_counts())


Independent         18480
CompanySponsored    15705
Name: AFFILIATION, dtype: int64


In [ ]:
#USE_CASE - Use case for funding
print(df['USE_CASE'].value_counts().head(10))

Preservation     27997
ProductDev        5659
CommunityServ      383
Heathcare          143
Other                3
Name: USE_CASE, dtype: int64


In [ ]:
# List of values to be binned as 'Other' for USE_CASE column
other_use_cases = ['CommunityServ', 'Heathcare', 'Other']

# Use the `replace` function to bin the desired values
df['USE_CASE'] = df['USE_CASE'].replace(other_use_cases, 'Other')

# Verify the changes
print(df['USE_CASE'].value_counts())


Preservation    27997
ProductDev       5659
Other             529
Name: USE_CASE, dtype: int64


In [ ]:
#ORGANIZATION - Organization type
print(df['ORGANIZATION'].value_counts().head(10))

Trust           23418
Association     10243
Co-operative      483
Corporation        41
Name: ORGANIZATION, dtype: int64


In [ ]:
# List of values to be binned as 'Other' for ORGANIZATION column
#other_organizations = ['Co-operative', 'Corporation']

# Use the `replace` function to bin the desired values
#df['ORGANIZATION'] = df['ORGANIZATION'].replace(other_organizations, 'Other')

#drop everything except trust and assoc.
df = df[df['ORGANIZATION'].isin(['Trust', 'Association'])]

# Verify the changes
print(df['ORGANIZATION'].value_counts())


Trust          23418
Association    10243
Name: ORGANIZATION, dtype: int64


In [ ]:
#STATUS - Active status
print(df['STATUS'].value_counts().head(10))

1    33656
0        5
Name: STATUS, dtype: int64


In [ ]:
#STATUS seems like it might not provide much predictive power given that almost all values are '1' and only a tiny fraction are '0'. It's essentially a near-constant column.
df = df.drop(columns=['STATUS'])

In [ ]:
#INCOME_AMT - Income classification
print(df['INCOME_AMT'].value_counts().head(10))

0                24144
25000-99999       3671
100000-499999     3279
1M-5M              868
1-9999             698
10000-24999        517
10M-50M            209
5M-10M             155
50M+               120
Name: INCOME_AMT, dtype: int64


In [ ]:
#SPECIAL_CONSIDERATIONS - Special considerations for application
print(df['SPECIAL_CONSIDERATIONS'].value_counts().head(10))

N    33634
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64


In [ ]:
#SPECIAL_CONSIDERATION is also heavily imbalanced and will be dropped for this test (3rd results, not dropped for 4th results)
#df = df.drop(columns=['SPECIAL_CONSIDERATIONS'])

In [ ]:
#ASK_AMT - Funding amount requested
print(df['ASK_AMT'].value_counts().head(20))

5000      25142
63981         3
15583         3
6725          3
10478         3
32141         2
146824        2
31453         2
7003          2
46327         2
63520         2
10070         2
6035          2
16071         2
42178         2
27073         2
5170          2
34875         2
21346         2
20537         2
Name: ASK_AMT, dtype: int64


In [ ]:
#def bin_ask_amt(value):
    #if value <= 10000:
        #return '0-10,000'
    #elif 10000 < value <= 50000:
        #return '10,001-50,000'
    #elif 50000 < value <= 100000:
        #return '50,001-100,000'
    #elif 100000 < value <= 500000:
        #return '100,001-500,000'
    #elif 500000 < value <= 1000000:
        #return '500,001-1,000,000'
    #else:
        #return '1,000,000+'

# Replace the ASK_AMT column with binned values
#df['ASK_AMT'] = df['ASK_AMT'].apply(bin_ask_amt)

# Adding the binary column
df['IS_5000'] = df['ASK_AMT'].apply(lambda x: 1 if x == 5000 else 0)

# Log transform (add 1 to avoid log(0))
df['ASK_AMT'] = np.log1p(df['ASK_AMT'])

# Min-Max scaling
scaler = MinMaxScaler()
df['ASK_AMT'] = scaler.fit_transform(df[['ASK_AMT']])

df['ASK_AMT']

0        0.000000
2        0.000000
3        0.020298
4        0.233353
5        0.000000
           ...   
34293    0.000000
34294    0.000000
34295    0.000000
34296    0.000000
34297    0.000000
Name: ASK_AMT, Length: 33661, dtype: float64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_encoded = pd.get_dummies(df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "ASK_AMT", "SPECIAL_CONSIDERATIONS"])

In [ ]:
application_encoded

,IS_SUCCESSFUL,IS_5000,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ASK_AMT_0.8856751421231829,ASK_AMT_0.8946093997244929,ASK_AMT_0.9070634326610939,ASK_AMT_0.9084687625543199,ASK_AMT_0.9352172049089761,ASK_AMT_0.9572350685987973,ASK_AMT_0.9700332293792021,ASK_AMT_0.9999999999999999,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34293,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34294,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


**Train, Compile, Etc.**

In [ ]:
def build_model(hp):
    model = tf.keras.models.Sequential()

    # First hidden layer with a searchable number of units
    model.add(tf.keras.layers.Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32),
                                 activation='relu', input_dim=X_train_scaled.shape[1]))

    # Second hidden layer with a searchable number of units
    model.add(tf.keras.layers.Dense(units=hp.Int('units_2', min_value=32, max_value=512, step=32),
                                 activation='relu'))

    # Add more layers or hyperparameters here as needed

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model with a searchable learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_encoded["IS_SUCCESSFUL"].values
X = application_encoded.drop("IS_SUCCESSFUL", 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

<ipython-input-27-40dd3ad7175d>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_encoded.drop("IS_SUCCESSFUL", 1).values


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=40,
    directory='hyperband_results',
    project_name='alphabet_soup_optimization'
)


NameError: ignored

In [ ]:
tuner.search(X_train_scaled, y_train, epochs=40, validation_data=(X_test_scaled, y_test))


Trial 58 Complete [00h 02m 52s]
val_accuracy: 0.7224334478378296

Best val_accuracy So Far: 0.7264733910560608
Total elapsed time: 02h 22m 36s

Search: Running Trial #59

Value             |Best Value So Far |Hyperparameter
480               |512               |units_1
224               |448               |units_2
0.01              |0.001             |learning_rate
5                 |40                |tuner/epochs
0                 |14                |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/5
789/789 [==============================] - 54s 68ms/step - loss: 0.6312 - accuracy: 0.6902 - val_loss: 0.6040 - val_accuracy: 0.7087
Epoch 2/5
 97/789 [==>...........................] - ETA: 36s - loss: 0.5499 - accuracy: 0.7400

KeyboardInterrupt: ignored

**Results of Hyperband for base set - 73.38%(no changes from inital dataset other than binning). This data set worked best:**</br>
Value             | Best Value So Far | Hyperparameter</br>
448               | 480               | units_1</br>
288               | 64                | units_2</br>
0.0001            | 0.001             | learning_rate</br>
14                | 14                | tuner/epochs</br>
0                 | 5                 | tuner/initial_epoch</br>
1                 | 3                 | tuner/bracket</br>
0                 | 2                 | tuner/round</br>

**Results of Hyperband for second set - 72.3%(dropped Status, Special Considerations and binned ASK_AMT):**</br>
Value             | Best Value So Far | Hyperparameter</br>
480               | 416               | units_1</br>
352               | 480               | units_2</br>
0.0001            | 0.001             | learning_rate</br>
40                | 5                 | tuner/epochs</br>
14                | 2                 | tuner/initial_epoch</br>
3                 | 3                 | tuner/bracket</br>
3                 | 1                 | tuner/round</br>
0047              | 0010              | tuner/trial_id</br>

**Results of Hyperband for third set - 73.3%(dropped Status, Special Considerations and binned ASK_AMT, ORGANIZATION, USE_CASE, AFFILIATION ):**</br>
Value             | Best Value So Far | Hyperparameter</br>
192               | 480               | units_1</br>
160               | 96                | units_2</br>
0.0001            | 0.001             | learning_rate</br>
5                 | 14                | tuner/epochs</br>
0                 | 5                 | tuner/initial_epoch</br>
2                 | 3                 | tuner/bracket</br>
0                 | 2                 | tuner/round</br>

**Results of Hyperband for final set - 72.6%(dropped Status,  kept Special Considerations and dropped/binned ASK_AMT, ORGANIZATION, USE_CASE, AFFILIATION ) This set had awful loss:**</br>
Value             | Best Value So Far | Hyperparameter</br>
480               | 512               | units_1</br>
224               | 448               | units_2</br>
0.01              | 0.001             | learning_rate</br>
5                 | 40                | tuner/epochs</br>
0                 | 14                | tuner/initial_epoch</br>
2                 | 3                 | tuner/bracket</br>
0                 | 3                 | tuner/round</br>


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
best_model.fit(X_train_scaled, y_train, epochs=40, validation_data=(X_test_scaled, y_test))


Epoch 1/40
789/789 [==============================] - 63s 78ms/step - loss: 0.3943 - accuracy: 0.8129 - val_loss: 0.6920 - val_accuracy: 0.7260
Epoch 2/40
789/789 [==============================] - 72s 91ms/step - loss: 0.3941 - accuracy: 0.8117 - val_loss: 0.6740 - val_accuracy: 0.7231
Epoch 3/40
789/789 [==============================] - 69s 88ms/step - loss: 0.3947 - accuracy: 0.8126 - val_loss: 0.6756 - val_accuracy: 0.7248
Epoch 4/40
789/789 [==============================] - 75s 95ms/step - loss: 0.3940 - accuracy: 0.8127 - val_loss: 0.6884 - val_accuracy: 0.7218
Epoch 5/40
789/789 [==============================] - 67s 85ms/step - loss: 0.3943 - accuracy: 0.8126 - val_loss: 0.7674 - val_accuracy: 0.7249
Epoch 6/40
789/789 [==============================] - 70s 88ms/step - loss: 0.3963 - accuracy: 0.8128 - val_loss: 0.8102 - val_accuracy: 0.7227
Epoch 7/40
789/789 [==============================] - 72s 92ms/step - loss: 0.3936 - accuracy: 0.8136 - val_loss: 0.8058 - val_accuracy:

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
#nn.add(tf.keras.layers.Dense(units=416, activation="relu", input_dim=X_train_scaled.shape[1])) #old value from test 2
#nn.add(tf.keras.layers.Dense(units=480, activation="relu", input_dim=X_train_scaled.shape[1])) #old value from test 3
nn.add(tf.keras.layers.Dense(units=512, activation="relu", input_dim=X_train_scaled.shape[1]))

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=480, activation="relu")) #old value from test 2
#nn.add(tf.keras.layers.Dense(units=96, activation="relu")) #old value form test 3
nn.add(tf.keras.layers.Dense(units=448, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               4303872   
                                                                 
 dense_4 (Dense)             (None, 448)               229824    
                                                                 
 dense_5 (Dense)             (None, 1)                 449       
                                                                 
Total params: 4,534,145
Trainable params: 4,534,145
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Save the model's weights every 5 epochs
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5*len(X_train_scaled))

In [ ]:
#Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint])

Epoch 1/100
789/789 [==============================] - 47s 58ms/step - loss: 0.6049 - accuracy: 0.6929
Epoch 2/100
789/789 [==============================] - 52s 65ms/step - loss: 0.4144 - accuracy: 0.8079
Epoch 3/100
789/789 [==============================] - 50s 63ms/step - loss: 0.4059 - accuracy: 0.8086
Epoch 4/100
789/789 [==============================] - 50s 63ms/step - loss: 0.4029 - accuracy: 0.8086
Epoch 5/100
789/789 [==============================] - 50s 64ms/step - loss: 0.4012 - accuracy: 0.8086
Epoch 6/100
789/789 [==============================] - 48s 61ms/step - loss: 0.4012 - accuracy: 0.8096
Epoch 7/100
789/789 [==============================] - 50s 64ms/step - loss: 0.3996 - accuracy: 0.8099
Epoch 8/100
789/789 [==============================] - 49s 62ms/step - loss: 0.3997 - accuracy: 0.8096
Epoch 9/100
789/789 [==============================] - 49s 62ms/step - loss: 0.3998 - accuracy: 0.8108
Epoch 10/100
789/789 [==============================] - 51s 65ms/step - l

KeyboardInterrupt: ignored

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

263/263 - 3s - loss: 0.9115 - accuracy: 0.7151 - 3s/epoch - 10ms/step
Loss: 0.9114940762519836, Accuracy: 0.7150665521621704


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")